In [8]:
#Members: Luyao Wang, Di Jin, Yingqi Lin

In [12]:
import re
def remove_special_characters(word):
    regex = re.compile('[^a-zA-Z]')
    return regex.sub('', word)

list2=[] #two-dimensional list

def read_data(filename):
    global list2
    i=0 # total number of rows
    j=0 # total number of columns
    
    f=open(filename,'r')
    list2=f.read()
    list2=list2.split("\n")
    for i in range(len(list2)):
        list2[i]=list2[i].split()
    for i in range(len(list2)):
        for j in range(len(list2[i])):
            list2[i][j]=remove_special_characters(list2[i][j])
    return list2

#get the column number of an attribut
def find_item(list1,attribute):
    item=0 #对应的列数
    while item < len(list1[0]):
        if attribute==list1[0][item]:
            break
        else:
            item=item+1
    return item

#get a list only contains the unique values of an attribute
def distinct_words(list1,attribute):    
        #find the column number of this attribute
        item=find_item(list1,attribute)    
        #change the column to the One-dimensional list
        k=1
        text=[]
        while k<len(list1):
            text.append(list1[k][item])
            k=k+1    
        #put the unique values into text_final[]
        text_final = []
        for t in text:
            if t not in text_final:
                text_final.append(t)
        return text_final

In [13]:
#Note: in the file "data.txt", we change one of the original attribute’s name “Favorite Beer” into “FavotiteBeer”

In [17]:
import pdb
import pandas as pd 
from math import log 
from anytree import Node, RenderTree
from anytree.dotexport import RenderTreeGraph
import fractions
import math

def create_decision_tree_id3(list1, y_col):

    def weight(list1,attribute,value):#return a fraction    
        #get the column number of an attribut
        item=find_item(list1,attribute)    
        #how many values
        k=1 #loop variable
        count=0 #number of values
        while k<len(list1):
            if list1[k][item]==value: 
                count=count+1
            k=k+1
        w=fractions.Fraction(count,len(list1)-1)
        return w
    
    
    def possibility(list1,attribute,value,lable_value):#return a fraction 
         #get the column number of an attribut
        item=find_item(list1,attribute)    
        #how many values and how many "yes"/"No"
        k=1 #loop variable
        count=0 #number of values
        count_lable=0 #number of "yes" or "no"
        while k<len(list1):
            if list1[k][item]==value:
                count=count+1
                if list1[k][-1]==lable_value:
                    count_lable=count_lable+1
            k=k+1
        p=fractions.Fraction(count_lable,count)
        return p
    
    def entropy_D(list1):#entropy of root
        p=weight(list1,'Enjoy','Yes')
        entropy= -p*(math.log(p,2))-(1-p)*(math.log(1-p,2))
        return entropy

    def entropy_D1(p):# entropy of nonroot
        if p==1 or p==0:
            entropy= 0
        else:
            entropy= -p*(math.log(p,2))-(1-p)*(math.log(1-p,2))
        return entropy   
    
    
    def entropy_A_D(list1,attribute):#average entropy
        txt = distinct_words(list1,attribute) 
        en = [] #entropies of every values in a particular attribute
        for t in txt:
            p=possibility(list1,attribute,t,'Yes')
            en_p=entropy_D1(p)
            w=weight(list1,attribute,t)
            en_w=w*en_p
            en.append(en_w)        
        return sum(en)


    def get_max_info_gain(list1,y_col): # return the name of attribute 
        d = {}  
        h = entropy_D(list1)
        for c in list1[0]:
            #get the column number of an attribut
            item=find_item(list1,c)        
            if c!=y_col:
                if list1[1][item]!='': 
                    d[c] = h - entropy_A_D(list1,c)
        if not d:
            pass #d is empty
        else:
            return max(d,key=d.get) 


     
    def train_decision_tree(node, list1, y_col):
        c = get_max_info_gain(list1, y_col) 
        if c!=None:# only have the label：enjoy
            for v in distinct_words(list2,c):
                if v in distinct_words(list2,c) and v not in distinct_words(list1,c):
                    # count the number of "yes"/"no"
                    k=1 #control the loop
                    count_lable_y=0
                    count_lable_n=0
                    while k<len(list1):
                        if list1[k][-1]=='Yes':
                             count_lable_y=count_lable_y+1
                        else:
                            count_lable_n=count_lable_n+1
                        k=k+1
                    if count_lable_y>=count_lable_n:
                        curr_node=Node('%s %s'%(c,v), parent=node)
                        Node('%s %s'%('Enjoy','Yes'), parent=curr_node)
                    else:
                        curr_node=Node('%s %s'%(c,v), parent=node)
                        Node('%s %s'%('Enjoy','No'), parent=curr_node)
                else:
                    if v=='':
                        continue
                    else:
                        p=possibility(list1,c,v,'Yes')
                        curr_node = Node('%s %s'%(c,v),parent=node)
                        # if the attributes are not used up
                        if (len(list1[1])-list1[1].count('')) > 2:                    
                            if p == 1:# if the purity is 100%, a leaf node "Yes" is generated 
                                Node('Enjoy Yes', parent=curr_node)
                            elif p == 0: #if the purity is 100%, a leaf node "No" is generated 
                                Node('Enjoy No', parent=curr_node)            
                            else:  #not pure 
                                #get the column number of an attribut
                                item=find_item(list1,c)                                
                                #withsraw a sub list and replace the column of attribute "c" with ''
                                k=1
                                sub_list1=[]
                                sub_list1.append(list1[0])
                                while k<len(list1):
                                    if(v==list1[k][item]):
                                        sub_list1.append(list1[k])
                                        sub_list1[-1][item]= sub_list1[-1][item].replace(sub_list1[-1][item],'')
                                    k=k+1
                                train_decision_tree(curr_node, sub_list1, y_col)                        
                        #if the attributs run out                                                
                        else:
                            q=1
                            while q<len(list1[1])-1:
                                if list1[1][q]!='':
                                    Node('Enjoy No', parent=curr_node)
                                q=q+1
    
    root_node = Node('root')
    train_decision_tree(root_node, list1, y_col)
    return root_node


In [18]:
data=read_data('dt_data.txt') 
root_node = create_decision_tree_id3(data, 'Enjoy')
for pre, fill, node in RenderTree(root_node):
    print("%s%s" % (pre, node.name))
RenderTreeGraph(root_node).to_picture("decision_tree_id3.png")

root
├── Occupied High
│   ├── Location Talpiot
│   │   └── Enjoy No
│   ├── Location CityCenter
│   │   └── Enjoy Yes
│   ├── Location GermanColony
│   │   └── Enjoy No
│   ├── Location EinKarem
│   │   └── Enjoy Yes
│   └── Location MahaneYehuda
│       └── Enjoy Yes
├── Occupied Moderate
│   ├── Location Talpiot
│   │   ├── Price Expensive
│   │   │   └── Enjoy Yes
│   │   ├── Price Normal
│   │   │   └── Enjoy Yes
│   │   └── Price Cheap
│   │       └── Enjoy No
│   ├── Location CityCenter
│   │   └── Enjoy Yes
│   ├── Location GermanColony
│   │   ├── VIP No
│   │   │   └── Enjoy No
│   │   └── VIP Yes
│   │       └── Enjoy Yes
│   ├── Location EinKarem
│   │   └── Enjoy Yes
│   └── Location MahaneYehuda
│       └── Enjoy Yes
└── Occupied Low
    ├── Location Talpiot
    │   └── Enjoy No
    ├── Location CityCenter
    │   ├── Price Expensive
    │   │   └── Enjoy No
    │   ├── Price Normal
    │   │   ├── Music Loud
    │   │   │   └── Enjoy Yes
    │   │   └── Music Quiet
    │

In [19]:
import copy
def predict(root_node, test, label):
    global list2
    root = copy.deepcopy(root_node)
    root.children[0].name=root.children[0].name.split(' ')
    if root.children[0].name[0]!=label:
        item=find_item(list2,root.children[0].name[0])
        i=0
        while i<len(root.children):
            if i!=0:
                root.children[i].name=root.children[i].name.split(' ')
            #print(i,len(root.children),test[item-1],root.children[i].name[1])
            if test[item-1]==root.children[i].name[1]:
                next_root=root.children[i]
                return predict(next_root,test,label)
            i=i+1
        return root.children[0].name[1]
    else:
        return root.children[0].name[1]


In [20]:
df=read_data('dt_data.txt') 
root_node = create_decision_tree_id3(df, 'Enjoy') 
print(predict(root_node,['Moderate','Cheap','Loud','CityCenter','No','No'],'Enjoy'))
print(predict(root_node,['Low','Normal','Quiet','CityCenter','No','No'],'Enjoy'))

Yes
No
